In [0]:
import numpy
import random
from time import time

In [0]:
class Constants:
  
  SIGMOID = "sigmoid"

In [0]:
class Neuron:
  
  # id_ = layer.neuron_num
  id_ = None
  # name of the activation function
  act_func = None 
  in_edges = None
  out_edges = None
  # output of a neuron i.e activation(summation(w_i*x_i))
  output = None       
  # error at a neuron during back propagation
  local_error = None  
  # input to the input-layer-neuron
  input_datum = None   
  # derivative of loss; loss = softmax and then cross entropy
  loss_der = None   

In [0]:
class Neuron(Neuron):
  
  # returns output of activation function
  def apply_act_func(self, inp):
    if(self.act_func == Constants.SIGMOID):
      return ( 1/ ( 1+numpy.exp(-1*inp) ) )

In [0]:
class Neuron(Neuron):
  
  # returns derivative of activation function
  def apply_act_func_der(self, inp):
    if(self.act_func == Constants.SIGMOID):
      return (inp*(1-inp))

In [0]:
class Neuron(Neuron):
  
  # computes output of a neuron
  # assumption-1: outputs of start_neurons of all self.in_edges are calculated
  # assumption-2: self.input_datum is initialized for a first-layered-neuron
  def feed_forward(self):
    # first-layered-neuron
    if(self.in_edges == None): 
      self.output = self.input_datum
      return
    output = 0.0
    ind = 0
    # compute summation(w_i*x_i)
    while(ind < len(self.in_edges)):
      edge = self.in_edges[ind]
      weight = edge.weight
      inp = edge.start_neuron.output
#       if(self.out_edges == None):
#         print("edge - wi*xi: " + str(edge) + " - " + str(weight) + ", " + str(inp))
      output = output + (weight*inp)
      ind = ind + 1
    # for output-layered-neurons, softmax should be applied
    if(self.out_edges != None):
      # apply activation function
      self.output = self.apply_act_func(output)
    else:
      self.output = output
#       print("summation(wi*xi): " + str(output))

In [0]:
class Neuron(Neuron):
  
  # back propagation (updates the weights of in_edges)
  # local_error: before multiplying by weight
  # assumption-1: local_errors of all end_neurons of self.out_edges are calculated
  # assumption-2: self.output is computed
  # assumption-3: self.loss_der is initialized for an output-layered-neuron
  def back_prop(self):
    # output-layered-neuron
    if(self.out_edges == None): 
      self.local_error = self.loss_der
    else:
      ind = 0
      local_error_self = 0.0
      # compute self.local_error
      while(ind < len(self.out_edges)):
        edge = self.out_edges[ind]
        weight = edge.old_weight
        local_error_other = edge.end_neuron.local_error
        local_error_self = local_error_self + (local_error_other * weight)
        ind = ind + 1
      act_func_der = self.apply_act_func_der(self.output)
      self.local_error = local_error_self * act_func_der
    # update self.in_edges
    if(self.in_edges != None):
      ind = 0
      while(ind < len(self.in_edges)):
        edge = self.in_edges[ind]
        inp = edge.start_neuron.output
        update = self.local_error * inp
        edge.old_weight = edge.weight
        edge.weight = edge.weight + update
        ind = ind + 1

In [0]:
# An edge between 2 neurons
class Edge:
  
  id_ = None
  start_neuron = None
  end_neuron = None
  weight = None
  # old weight is required during back propagation i.e out_edges' weights are already 
  # updated, but you need old weights, Why? During back propagation, weight updations 
  # are done from right to left; at any instance during back propagation, all the 
  # weights of edges that are towards right are already updated
  old_weight = None

In [0]:
class Edge(Edge):
  
  def __str__(self):
    return str(self.start_neuron.id_ + "->" + self.end_neuron.id_)

In [0]:
# Fully connected Layer
class Layer:
  
  # id_ = layer_num
  id_ = None
  tot_neurons = None
  act_func = None
  neurons = None
  prev_layer = None
  next_layer = None

In [0]:
class Layer(Layer):
  
  # update in_edges of cur_layer's neurons & out_edges of prev_layer's neurons
  # assumption: should have non-None value to self.prev_layer from 2nd layer onwards
  def build(self):
    # cur refers to current layer
    cur_neuron_ind = 0 
    self.neurons = []
    # add neurons
    while(cur_neuron_ind < self.tot_neurons):
      cur_neuron = Neuron()
      cur_neuron.id_ = self.id_ + "." + str(cur_neuron_ind)
      cur_neuron.act_func = self.act_func
      # add in_edges to cur_neuron and out_edges to prev_neuron
      if(self.prev_layer != None):
        cur_neuron.in_edges = []
        prev_neuron_ind = 0
        while(prev_neuron_ind < self.prev_layer.tot_neurons):
          prev_neuron = self.prev_layer.neurons[prev_neuron_ind]
          if(prev_neuron.out_edges == None):
            prev_neuron.out_edges = []
          edge = Edge()
          edge.id_ = str(prev_neuron.id_) + "." + str(len(prev_neuron.out_edges))
          edge.start_neuron = prev_neuron
          edge.end_neuron = cur_neuron
          edge.weight = random.uniform(0, 1)
          edge.weight = round(edge.weight, 1)
          cur_neuron.in_edges.append(edge)
          prev_neuron.out_edges.append(edge)
          prev_neuron_ind = prev_neuron_ind + 1
      self.neurons.append(cur_neuron)
      cur_neuron_ind = cur_neuron_ind + 1

In [0]:
class Layer(Layer):
  
  def feed_forward(self):
    neuron_ind = 0
    while(neuron_ind < self.tot_neurons):
      neuron = self.neurons[neuron_ind]
      neuron.feed_forward()
      neuron_ind = neuron_ind + 1

In [0]:
class Layer(Layer):
  
  def back_prop(self):
    neuron_ind = 0
    while(neuron_ind < self.tot_neurons):
      neuron = self.neurons[neuron_ind]
      neuron.back_prop()
      neuron_ind = neuron_ind + 1

In [0]:
class Neural_Network:
  
  # including input, hidden and output layers
  tot_layers = None
  # a list containing no. of neurons in each layer
  tot_neurons = None
  # single activation func; ex: Constants.SIGMOID
  act_func = None
  # a list containing actual layers
  layers = None
  train_x = None
  train_y = None
  # Index of sample that is being feed_forwarded or backpropagated
  sample_ind = None
  # total softmax loss after forward propagation
  #tot_loss = None
  tot_epochs = None

In [0]:
class Neural_Network(Neural_Network):
  
  def build(self):
    cur_layer_ind = 0
    prev_layer_ind = -1
    self.layers = []
    while(cur_layer_ind < self.tot_layers):
      cur_layer = Layer()
      cur_layer.id_ = str(cur_layer_ind)
      cur_layer.tot_neurons = self.tot_neurons[cur_layer_ind]
      cur_layer.act_func = self.act_func
      if(prev_layer_ind != -1):
        cur_layer.prev_layer = self.layers[prev_layer_ind]
      cur_layer.build()
      self.layers.append(cur_layer)
      cur_layer_ind = cur_layer_ind + 1
      prev_layer_ind = prev_layer_ind + 1

In [0]:
class Neural_Network(Neural_Network):
  
  def print(self):
    print("layers: " + str(self.tot_layers))
    print("neurons: " + str(self.tot_neurons))
    print("activation: " + str(self.act_func))
    print("=======================")
    print("layer.neuron.edge: weight")
    layer_ind = 0
    while(layer_ind < self.tot_layers):
      cur_layer = self.layers[layer_ind]
      neuron_ind = 0
      while(neuron_ind < cur_layer.tot_neurons):
        cur_neuron = cur_layer.neurons[neuron_ind]
        if(cur_neuron.output != None):
          cur_output = str(round(cur_neuron.output, 2))
        else:
          cur_output = "None"
        if(cur_neuron.loss_der != None):
          cur_loss_der = str(round(cur_neuron.loss_der, 2))
        else:
          cur_loss_der = "None"
        print(cur_output + "|" + cur_loss_der)
        if(cur_neuron.out_edges != None):
          out_edge_ind = 0
          while(out_edge_ind < len(cur_neuron.out_edges)):
            out_edge = cur_neuron.out_edges[out_edge_ind]
            print(str(out_edge.id_) + ": ", end="")
            print(str(round(out_edge.weight, 2)))
            out_edge_ind = out_edge_ind + 1
        neuron_ind = neuron_ind + 1
      layer_ind = layer_ind + 1
      if(layer_ind != self.tot_layers):
        print("=======================")
    print("=====================================================")

In [0]:
class Neural_Network(Neural_Network):
  
  # feeds sample to the 1st layer
  # i.e for each neuron in the 1st layer, a feature (of a sample) should be assigned to its neuron.input_datum
  # assumption: No. of neurons in 1st layer = no. of features in a sample
  def feed_sample(self, sample):
    layer_one = self.layers[0]
    neurons = layer_one.neurons
    neuron_ind = 0
    while(neuron_ind < layer_one.tot_neurons):
      neuron = neurons[neuron_ind]
      feature = sample[neuron_ind]
      neuron.input_datum = feature
      neuron_ind = neuron_ind + 1

In [0]:
class Neural_Network(Neural_Network):
  
  def softmax(self, data):
    ind = 0
    exp_data = []
    while(ind < len(data)):
      cur_exp = numpy.exp(data[ind])
      exp_data.append(cur_exp)
      ind = ind + 1
    sum_exp = sum(exp_data)
    softmax_data = []
    ind = 0
    while(ind < len(exp_data)):
      cur_softmax = exp_data[ind] / sum_exp
      softmax_data.append(cur_softmax)
      ind = ind + 1
    return softmax_data

In [0]:
class Neural_Network(Neural_Network):

  # feeds a sample to the 1st layer
  def feed_forward(self):
    sample = self.train_x[self.sample_ind]
    #print("input: " + str(sample))
    self.feed_sample(sample)
    layer_ind = 0
    while(layer_ind < self.tot_layers):
      cur_layer = self.layers[layer_ind]
      cur_layer.feed_forward()
      layer_ind = layer_ind + 1
    # softmax for output layer
    output_layer = self.layers[self.tot_layers-1]
    neuron_ind = 0
    # output of each neuron
    outputs = []
    while(neuron_ind < output_layer.tot_neurons):
      neuron = output_layer.neurons[neuron_ind]
      cur_output = neuron.output
      outputs.append(cur_output)
      neuron_ind = neuron_ind + 1
    softmax_outputs = self.softmax(outputs)
    # modify outputs of each neuron in output layer to a softmax output
    neuron_ind = 0
    while(neuron_ind < output_layer.tot_neurons):
      neuron = output_layer.neurons[neuron_ind]
      neuron.output = softmax_outputs[neuron_ind]
      neuron_ind = neuron_ind + 1

In [0]:
class Neural_Network(Neural_Network):
  
  # Softmax and then Cross Entropy Loss
  # assumption-1: class labels range between 0 and tot_classes-1
  def compute_loss(self):
    last_layer = self.layers[self.tot_layers-1]
    neurons = last_layer.neurons
    cur_label = self.train_y[self.sample_ind]
    # derivative of loss
    neuron_ind = 0
    while(neuron_ind < len(neurons)):
      neuron = neurons[neuron_ind]
      if(neuron_ind == cur_label):
        ground_truth_softmax = 1
      else:
        ground_truth_softmax = 0
      predicted_softmax = neuron.output
      neuron.loss_der = predicted_softmax - ground_truth_softmax
      neuron_ind = neuron_ind + 1
    # compute total loss (just for observing loss over iterations)
    neuron_ind = 0
    self.total_loss = 0.0
    while(neuron_ind < len(neurons)):
      softmax_output = neurons[neuron_ind].output
      if(neuron_ind == cur_label):
        ground_truth_softmax = 1
      else:
        ground_truth_softmax = 0
      print(str(round(softmax_output, 1)) + " | " + str(ground_truth_softmax))
#       softmax_loss = ground_truth_softmax * numpy.log(softmax_output)
#       self.total_loss = self.total_loss + softmax_loss
      neuron_ind = neuron_ind + 1
    self.total_loss = -1 * self.total_loss
    

In [0]:
class Neural_Network(Neural_Network):
  
  def back_prop(self):
    layer_ind = self.tot_layers - 1
    while(layer_ind >= 0 ):
      layer = self.layers[layer_ind]
      layer.back_prop()
      layer_ind = layer_ind - 1

In [0]:
class Neural_Network(Neural_Network):
  
  def train(self):
    self.build()
    cur_epoch = 0
    while(cur_epoch < self.tot_epochs):
      self.sample_ind = 0
      while(self.sample_ind < len(self.train_x)):
        self.feed_forward()
        self.compute_loss()
        #self.print()
        self.back_prop()
        #self.print()
        #return
        self.sample_ind = self.sample_ind + 1
        #print("*******************sample************************")
      print("================")
      cur_epoch = cur_epoch + 1

In [0]:
class Main:
  
  @staticmethod
  def main():
    train_x = [[0, 0], [0, 1], [1, 0], [1, 1]]
    train_y = [0, 1, 1, 0]
    nn = Neural_Network()
    nn.tot_layers = 3
    nn.tot_neurons = [2, 3, 2]
    nn.act_func = Constants.SIGMOID
    nn.train_x = train_x
    nn.train_y = train_y
    nn.tot_epochs = 9
    nn.train()

In [336]:
start_time = time()
Main.main()
end_time = time()
elapsed_time = end_time - start_time
print("processed in " + str(elapsed_time) + " seconds.")

0.5 | 1
0.5 | 0
0.3 | 0
0.7 | 1
0.6 | 0
0.4 | 1
0.9 | 1
0.1 | 0
0.7 | 1
0.3 | 0
0.7 | 0
0.3 | 1
0.9 | 0
0.1 | 1
1.0 | 1
0.0 | 0
1.0 | 1
0.0 | 0
1.0 | 0
0.0 | 1
1.0 | 0
0.0 | 1
1.0 | 1
0.0 | 0
1.0 | 1
0.0 | 0
1.0 | 0
0.0 | 1
1.0 | 0
0.0 | 1
1.0 | 1
0.0 | 0
1.0 | 1
0.0 | 0
1.0 | 0
0.0 | 1
1.0 | 0
0.0 | 1
1.0 | 1
0.0 | 0
1.0 | 1
0.0 | 0
1.0 | 0
0.0 | 1
1.0 | 0
0.0 | 1
1.0 | 1
0.0 | 0
1.0 | 1
0.0 | 0
1.0 | 0
0.0 | 1
1.0 | 0
0.0 | 1
1.0 | 1
0.0 | 0
1.0 | 1
0.0 | 0
1.0 | 0
0.0 | 1
1.0 | 0
0.0 | 1
1.0 | 1
0.0 | 0
1.0 | 1
0.0 | 0
1.0 | 0
0.0 | 1
1.0 | 0
0.0 | 1
1.0 | 1
0.0 | 0
processed in 0.01789689064025879 seconds.
